In [11]:
#importing the libraries

import pandas as pd
import yfinance as yf
import numpy as np

In [16]:
port_pick = pd.read_csv(r'Data\portfolio_pick.csv')['Stock'].tolist()

In [ ]:
def fetch_high_volatility_tickers(ticker_list, threshold=0.1):
    high_vol_tickers = []
    vol_data = {}

    for ticker in ticker_list:
        try:
            stock_data = yf.download(ticker, period="1y", interval="1d", progress=False)
            
            if stock_data.empty:
                print(f"No data found for {ticker}.")
                continue

            closing_prices = stock_data['Close']
            std_dev = np.std(closing_prices)
            mean_price = np.mean(closing_prices)

            vol_data[ticker] = {
                "std_dev": std_dev,
                "mean_price": mean_price,
                "std_dev_pct": std_dev / mean_price if mean_price > 0 else 0
            }

            if vol_data[ticker]["std_dev_pct"] >= threshold:
                high_vol_tickers.append(ticker)

            print(f"Processed {ticker}: Std Dev = {std_dev:.2f}, Mean Price = {mean_price:.2f}, Std Dev % = {std_dev / mean_price:.2%}")

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return high_vol_tickers, vol_data

if __name__ == "__main__":
    vol_tickers = port_pick

    threshold = 0.1

    high_vol_tickers, vol_data = fetch_high_volatility_tickers(port_pick, threshold)

    print("\nTickers with standard deviation >= 10%:")
    for ticker in high_vol_tickers:
        print(f"{ticker}: Std Dev % = {vol_data[ticker]['std_dev_pct']:.2%}")

In [19]:
def fetch_options_data(ticker_list):
    options_data = {}

    for ticker in ticker_list:
        try:
            stock = yf.Ticker(ticker)
            
            expiration_dates = stock.options
            
            ticker_data = {"calls": [], "puts": []}
            
            for date in expiration_dates:
                opt_chain = stock.option_chain(date)
                
                ticker_data["calls"].append({"expiration_date": date, "data": opt_chain.calls})
                ticker_data["puts"].append({"expiration_date": date, "data": opt_chain.puts})
            
            options_data[ticker] = ticker_data
            print(f"Fetched options data for {ticker}")
        
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    
    return options_data


if __name__ == "__main__":
    options_list = high_vol_tickers
    
    all_options_data = fetch_options_data(options_list)

Fetched options data for EBAY
Fetched options data for CMG
Fetched options data for EXPE
Fetched options data for GM
Fetched options data for DG
Fetched options data for AXP
Fetched options data for BAC
Fetched options data for BLK
Fetched options data for FIS
Fetched options data for GS
Fetched options data for MS
Fetched options data for WFC
Fetched options data for BIIB
Fetched options data for LMT
Fetched options data for AMD
Fetched options data for CRM
Fetched options data for MU
Fetched options data for QCOM


In [ ]:
def analyze_straddle_profit(ticker, std_dev, current_price):
    try:
        stock = yf.Ticker(ticker)
        expiration_dates = stock.options
        
        results = []

        for date in expiration_dates[:1]:
            opt_chain = stock.option_chain(date)
            calls = opt_chain.calls
            puts = opt_chain.puts
            
            relevant_calls = calls[np.abs(calls['strike'] - current_price) < std_dev]
            relevant_puts = puts[np.abs(puts['strike'] - current_price) < std_dev]
            
            for _, call in relevant_calls.iterrows():
                for _, put in relevant_puts.iterrows():
                    strike_price = call['strike']
                    call_price = call['lastPrice']
                    put_price = put['lastPrice']
                    straddle_cost = call_price + put_price
                    
                    upward_profit = max(0, (current_price + std_dev - strike_price) - call_price)
                    downward_profit = max(0, (strike_price - (current_price - std_dev)) - put_price)
                    
                    if upward_profit >= straddle_cost or downward_profit >= straddle_cost:
                        results.append({
                            "strike_price": strike_price,
                            "call_price": call_price,
                            "put_price": put_price,
                            "straddle_cost": straddle_cost,
                            "upward_profit": upward_profit,
                            "downward_profit": downward_profit,
                            "meets_criteria": True
                        })
        
        return results

    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")
        return []

def find_profitable_straddles(ticker_list, threshold=0.1):
    profitable_straddles = {}
    
    for ticker in ticker_list:
        try:
            stock_data = yf.download(ticker, period="1y", interval="1d", progress=False)
            closing_prices = stock_data['Close']
            std_dev = np.std(closing_prices)
            current_price = closing_prices.iloc[-1]

            if std_dev / current_price >= threshold:
                results = analyze_straddle_profit(ticker, std_dev, current_price)
                if results:
                    profitable_straddles[ticker] = results

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return profitable_straddles

if __name__ == "__main__":
    port_pick = high_vol_tickers
    threshold = 0.1
    profitable_straddles = find_profitable_straddles(port_pick, threshold)
    
    for ticker, straddles in profitable_straddles.items():
        print(f"\nProfitable Straddles for {ticker}:")
        for s in straddles:
            print(s)